## 「風しん抗体検査・風しん第５期定期接種受託医療機関」の可視化

「[厚生労働省　風しんの追加的対策について](https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/kenkou_iryou/kenkou/kekkaku-kansenshou/rubella/index_00001.html)」で公開されている「風しん抗体検査・風しん第５期定期接種受託医療機関」を地図上に可視化する．


PDFファイルから表データを取り出し，所在地の住所をアドレス・マッチングにより経緯度座標を割り出し，地図上にその位置を表示する．

## 利用したサービスやライブラリ

### 東京大学空間情報科学研究センター「CSVアドレスマッチングサービス」

http://newspat.csis.u-tokyo.ac.jp/geocode/modules/addmatch/index.php?content_id=1


### tabula

[tabula](https://tabula.technology/)は，PDFから表データを抽出するJavaアプリケーションです（Javaの実行環境が必要です）

### tabula-py

[tabula-py](https://pypi.org/project/tabula-py/) は，Pythonから tabula を利用して PDFの表データを pandas の DataFrame として取り出すことが可能なライブラリです

In [1]:
import tabula
import pandas as pd
import requests
from io import StringIO

In [2]:
# PDFファイルの URLを指定し，tabulaを使って表データを読み込み DataFrame にする
pdf_url = "https://www.mhlw.go.jp/content/000516970.pdf"

df = tabula.read_pdf(pdf_url, 
                     pages='all', # or 読み込むページ番号をリストで指定 ex. [1,2,3]
                     lattice=True, 
                     pandas_options={'header':[0,1]}
                    )

# カラム名を変更する
column_names = ['No', '実施機関名', '郵便番号', '所在地', '電話番号', '受託範囲：風しんの抗体検査', '受託範囲：風しんの第5期の定期接種']
df.columns = column_names

In [3]:
# 不要なカラムを削除
df.drop('No', axis='columns', inplace=True)

In [4]:
# 文字列のクレンジング

# 混在している◯と〇や，×を 1/0 に置換する
for col in ['受託範囲：風しんの抗体検査', '受託範囲：風しんの第5期の定期接種']:
    df[col] = df[col].fillna(0)
    df[col] = df[col].replace('×', 0)
    df[col] = df[col].replace('〇', 1).replace('○', 1)
    
# 値の中の改行除去
df['実施機関名'] = df['実施機関名'].str.replace('\r','')
df['所在地'] = df['所在地'].str.replace('`b\r','')

# 所在地の文字列からマンション名などを分離して住所のみ取り出す
df['所在地'] = df['所在地'].str.split(' ', expand=True)[0]

## アドレスマッチング・サービスで経緯度を割り出す

In [5]:
# DataFrameから所在地を文字列バッファにCSVデータにして出力する（エンコードは Shift_JISにしておく）
buffer = StringIO('csv-data')
df[['所在地']].to_csv(buffer, encoding='shift_jis', index=False, header=None)

In [6]:
# アドレスマッチング・サービスの設定
url_adress_matching = 'http://newspat.csis.u-tokyo.ac.jp/geocode-cgi/geocode.cgi'

params = {
    'action':'input',
    'spat_host':'newspat.csis.u-tokyo.ac.jp',
    'spat_port':8801, # 北海道，街区レベル（経緯度・世界測地系）
    'ncolumn': [1],   # カラム番号（先頭＝1）
    'input_kanji_code':'auto',
    'output_kanji_code':'auto',
    'exact_level':0
}

buffer.seek(0)

files = {'file': ("address.csv", buffer, 'text/csv')}

# アドレスマッチング・サービスにPOST
res = requests.post(url_adress_matching, files=files, data=params)
res.headers

{'Date': 'Fri, 02 Aug 2019 02:10:14 GMT', 'Server': 'Apache/2.2.15 (Red Hat)', 'Content-Disposition': 'attachment; filename=address.csv', 'Keep-Alive': 'timeout=15, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/octet-stream'}

In [7]:
# アドレスマッチングの結果のCSVデータを DataFrame にして，元の DataFrameに joinする
df_geodata = pd.read_csv( StringIO(res.text) )
df_result = df.join(df_geodata[['LocName', 'fX', 'fY', 'iConf', 'iLvl']])

In [8]:
# 経緯度のカラムをリネームする
df_result.rename({'fX':'Longitude', 'fY':'Latitude'}, axis='columns', inplace=True)

## kepler.glで可視化する

In [9]:
from keplergl import KeplerGl 
kepler = KeplerGl(height=500)
kepler.add_data(data=df_result, name="rubella")

kepler

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'rubella': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814